# Задача 2.3 Метод Холецкого

In [1]:
import math
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


def Cholesky_decomposition(A):
	assert np.linalg.eigvals(A).min() > 0  # checks if A is Positive Semidefinite
	n = A.shape[0]
	L = np.zeros((n,n), dtype=np.float64)
	for i in range(n): 
		for j in range(i + 1): 
			sum1 = 0
			if j == i: 
				for k in range(j):
					sum1 += pow(L[j][k], 2)
				L[j][j] = math.sqrt(A[j][j] - sum1)
			else:
				for k in range(j):
					sum1 += (L[i][k] * L[j][k])
				if L[j][j] > 0:
					L[i][j] = (A[i][j] - sum1) / L[j][j]

	return L


def LowerTriangular_solver(L, b):
	assert np.allclose(L, np.tril(L));  # checks if L is Lower Triangular 
	n = L.shape[0]
	y = np.zeros(n, dtype=np.float64)
	for i in range(n):
		tmp_sum = 0
		for j in range(i):
			tmp_sum += L[i][j] * y[j]
		y[i] = (b[i] - tmp_sum) / L[i][i]
		
	return y


def UpperTriangular_solver(U, y):
	assert np.allclose(U, np.triu(U));  # checks if L is Upper Triangular 
	return LowerTriangular_solver(np.flip(np.flip(U, 1), 0), y[::-1])[::-1]


def solve(A, b):
	assert np.allclose(A, A.T)
	n = A.shape[0]
	# A = L @ L.T
	L = Cholesky_decomposition(A)
	# L @ y = b
	y = LowerTriangular_solver(L, b)
	# U @ x = y
	U = L.T
	x = UpperTriangular_solver(U, y)
	return L, x

### Пример из Википедии https://en.wikipedia.org/wiki/Cholesky_decomposition

In [2]:
A = np.array([[4, 12, -16],
		      [12, 37, -43],
		      [-16, -43, 98]])
A

array([[  4,  12, -16],
       [ 12,  37, -43],
       [-16, -43,  98]])

Библиотечное разложение Холецкого

In [3]:
np.linalg.cholesky(A)

array([[ 2.,  0.,  0.],
       [ 6.,  1.,  0.],
       [-8.,  5.,  3.]])

Кастомное разложение Холецкого

In [4]:
Cholesky_decomposition(A)

array([[ 2.,  0.,  0.],
       [ 6.,  1.,  0.],
       [-8.,  5.,  3.]])

In [5]:
x = np.array([[3, 1, 1]]) # искомый вектор-решение
# Использую матричное умножение для проверки примера. Это не реализация требуемой функции
b = np.dot(A, x.T)
pd.DataFrame(b)

,0
0,8
1,30
2,7


Библиотечное решение системы

In [6]:
pd.DataFrame(np.linalg.solve(A, b))

,0
0,3.0
1,1.0
2,1.0


Кастомное решение системы

In [7]:
pd.DataFrame(solve(A, b)[1])

,0
0,3.0
1,1.0
2,1.0


### Теперь генерим положительно определённую матрицу рандомно: $A \cdot A^\top$

In [8]:
n = 5
A = np.random.rand(n, n).astype(np.float64)
# Использую матричное умножение для проверки примера. Это не реализация требуемой функции
A = np.dot(A, A.T)
A

array([[3.00122745, 2.46997351, 2.16489758, 1.51654762, 1.78850754],
       [2.46997351, 2.71103241, 2.28565905, 1.38786911, 1.65935825],
       [2.16489758, 2.28565905, 2.49368623, 1.31515801, 1.12747426],
       [1.51654762, 1.38786911, 1.31515801, 1.28742118, 1.11463908],
       [1.78850754, 1.65935825, 1.12747426, 1.11463908, 1.48630321]])

In [9]:
x = np.random.rand(n).astype(np.float64) # искомый вектор-решение
pd.DataFrame(x)

,0
0,0.516980
1,0.441090
2,0.759584
3,0.901290
4,0.640579


In [10]:
# Использую матричное умножение для проверки примера. Это не реализация требуемой функции
b = np.dot(A, x)
pd.DataFrame(b)

,0
0,6.798009
1,6.522711
2,5.929131
3,4.269528
4,4.469670


Библиотечное разложение Холецкого

In [11]:
npL = np.linalg.cholesky(A)
npL

array([[ 1.73240511,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 1.42574823,  0.82357416,  0.        ,  0.        ,  0.        ],
       [ 1.24964858,  0.61193616,  0.74672539,  0.        ,  0.        ],
       [ 0.87540011,  0.16971022,  0.15717114,  0.68380662,  0.        ],
       [ 1.03238413,  0.22759141, -0.404318  ,  0.34485251,  0.29375511]])

Кастомное разложение Холецкого

In [12]:
my_L = Cholesky_decomposition(A)
my_L

array([[ 1.73240511,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 1.42574823,  0.82357416,  0.        ,  0.        ,  0.        ],
       [ 1.24964858,  0.61193616,  0.74672539,  0.        ,  0.        ],
       [ 0.87540011,  0.16971022,  0.15717114,  0.68380662,  0.        ],
       [ 1.03238413,  0.22759141, -0.404318  ,  0.34485251,  0.29375511]])

Норма разности разложений Холецкого

In [13]:
np.linalg.norm(npL - my_L)

7.178989156685113e-16

Библиотечное решение системы

In [14]:
np_sol = np.linalg.solve(A, b)
pd.DataFrame(np_sol)

,0
0,0.516980
1,0.441090
2,0.759584
3,0.901290
4,0.640579


Кастомное решение системы

In [15]:
my_sol = solve(A, b)[1]
pd.DataFrame(my_sol)

,0
0,0.516980
1,0.441090
2,0.759584
3,0.901290
4,0.640579


Норма разности решений

In [16]:
np.linalg.norm(np_sol - my_sol)

4.030965858799381e-15